In [1]:
%matplotlib inline

In [2]:
from functools import partial

import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from matplotlib import pyplot as plt
from sklearn.pipeline import make_pipeline

In [4]:
SEED = 0
SEED_LIST = 2 ** np.array([2, 3, 5, 7, 11, 13, 17, 19, 23, 29])
VAL_SIZE = 0.3

In [5]:
train_csv = 'titanic/train.csv'
test_csv = 'titanic/test.csv'
submit_csv = 'titanic/submission.csv'

In [6]:
from sklearn.model_selection import RepeatedStratifiedKFold

def cv(params, n=100, n_cv=5, k=5):
    cv_results = xgb.cv(
        params,
        dfull,
        num_boost_round=n,
        folds=RepeatedStratifiedKFold(n_splits=k, n_repeats=n_cv, random_state=SEED),
        seed=SEED,
    )
    plot_cv(cv_results)
    return cv_results

def holdout(params, n=100, early_stopping_rounds=None):
    evals = {}
    m = xgb.train(
        params,
        dtrain,
        num_boost_round=n,
        evals=[(dtrain, 'train'), (dval, 'val')],
        evals_result=evals,
        early_stopping_rounds=early_stopping_rounds,
        verbose_eval=None,
    )
    plot_evals(evals)
    return evals

def train(params, n):
    return xgb.train(
        params,
        dfull,
        num_boost_round=n,
        verbose_eval=None,
    )

In [7]:
def roll(ls, w=5):
    return pd.Series(ls).rolling(window=w).mean()

def plot(a, b, c, d):
    plt.subplot(1, 2, 1)
    plt.plot(a), plt.plot(b)
    plt.ylim(0, 0.7)

    plt.subplot(1, 2, 2)
    plt.plot(c), plt.plot(d)
    plt.ylim(0, 0.2)

def plot_cv(cv_dict, start=0, stop=None):
    keys = [
        'train-logloss-mean',
        'test-logloss-mean',
        'train-error-mean',
        'test-error-mean'
    ]
    plot(*[roll(cv_dict[k][start:stop]) for k in keys])

def plot_evals(evals, start=0, stop=None):
    eval_list = [
        roll(evals[a][b][start:stop])
        for b in ['logloss', 'error']
        for a in ['train', 'val']
    ]
    plot(*eval_list)

def plot_cv_error(cv_results, start=0, stop=None):
    plt.plot(cv_results[['train-error-mean', 'test-error-mean']][start:stop])

def plot_holdout_error(h, start=0, stop=None):
    plt.plot(
        pd.DataFrame(
            [h['train']['error'], h['val']['error']],
            index=['train', 'val'])
        .T
        [start:stop]
    )

In [12]:
from sklearn.model_selection import train_test_split

def load(csv):
    ycol = 'target'

    col_names = {
        'Survived': ycol,
        'Pclass': 'ticket_class',
        'Name': 'name',
        'Sex': 'sex',
        'Age': 'age',
        'SibSp': 'n_sib_sp',
        'Parch': 'n_par_ch',
        'Ticket': 'ticket',
        'Fare': 'fare',
        'Cabin': 'cabin',
        'Embarked': 'port',
    }

    exclude = [
        'PassengerId'
    ]

    dtype = {
        'Pclass': np.uint8,
        'Age': np.float32,
        'SibSp': np.uint8,
        'Parch': np.uint8,
        'Fare': np.float32,
    }

    df = reorder(
        pd.read_csv(
            csv,
            dtype=dtype,
            usecols=lambda x: x not in exclude,
        )
        .rename(columns=col_names)
    )

    if ycol in df.columns:
        return df.drop(columns=ycol), df[ycol]
    else:
        return df

def load_titanic():
    X, y = load(train_csv)
    test = load(test_csv)
    traintest = pd.concat([X, test])
    return X, y, test, traintest

def preprocess(pip):
    full_X, full_y, todo_test, todo_traintest = load_titanic()

    todo_X, todo_val_X, y, val_y \
        = train_test_split(
            full_X,
            full_y,
            test_size=VAL_SIZE,
            stratify=full_y,
            random_state=SEED
        )

    tr_y = full_y
    tr_X = pip.fit_transform(full_X, full_y)
    traintest = pip.transform(todo_traintest)

    X = pip.fit_transform(todo_X, y)
    val_X = pip.transform(todo_val_X)
    test = pip.transform(todo_test)

    return (
        reorder(X), y,
        reorder(val_X), val_y,
        reorder(tr_X), tr_y,
        reorder(test), reorder(traintest)
    )

In [13]:
from sklearn.base import TransformerMixin


class Apply(TransformerMixin):
    def __init__(self, fn):
        self.fn = fn

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.apply(self.fn)


class AsType(TransformerMixin):
    def __init__(self, t):
        self.t = t

    def fit(self, X, y=None):
        if self.t == 'category':
            self.dtype = pd.Categorical(X.unique())
        else:
            self.dtype = self.t
        return self

    def transform(self, X):
        return X.astype(self.dtype)


class ColMap(TransformerMixin):
    def __init__(self, trf):
        self.trf = trf

    def fit(self, X, y=None):
        self.trf_list = [self.trf().fit(col) for _, col in X.iteritems()]
        return self
    
    def transform(self, X):
        cols = [t.transform(X.iloc[:, i]) for i, t in enumerate(self.trf_list)]
        return pd.concat(cols, axis=1)


class ColProduct(TransformerMixin):
    def __init__(self, trf):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.product(axis=1)


class ColQuot(TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.iloc[:, 0] / X.iloc[:, 1]


class ColSum(TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.sum(axis=1)


class Cut(TransformerMixin):
    def __init__(self, bins, interp=None):
        self.bins = bins
        self.interp = interp

    def fit(self, X, y=None):
        self.name = X.name
        self.vals = bin_interp(X, self.bins, self.interp)
        return self

    def transform(self, X):
        n = len(self.vals) - 2
        return (
            X
            .apply(lambda x: bin_val(x, self.bins, self.vals))
            .rename(f'{self.name}_cut{n}')
        )


class DataFrameUnion(TransformerMixin):
    def __init__(self, trf_list):
        self.trf_list = trf_list

    def fit(self, X, y=None):
        for t in self.trf_list:
            t.fit(X, y)
        return self

    def transform(self, X):
        return pd.concat([t.transform(X) for t in self.trf_list], axis=1)


class FillNA(TransformerMixin):
    def __init__(self, val):
        self.val = val

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.fillna(self.val)


class GetDummies(TransformerMixin):
    def __init__(self, drop_first=False):
        self.drop = drop_first

    def fit(self, X, y=None):
        self.name = X.name
        self.cat = pd.Categorical(X.unique())
        return self

    def transform(self, X):
        return pd.get_dummies(X.astype(self.cat), prefix=self.name, drop_first=self.drop)


class Identity(TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X


class Map(TransformerMixin):
    def __init__(self, d):
        self.d = d

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.map(self.d)


class MeanEncode(TransformerMixin):
    def __init__(self, y):
        self.y = y

    def fit(self, X, y=None):
        m = self.y.groupby(X).mean()
        keys = m.sort_values().index.values
        vals = m.index.values
        self.encode = {k: v for (k, v) in zip(keys, vals)}
        return self

    def transform(self, X):
        return X.replace(self.encode)


class NADummies(TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.isna().astype(np.uint8).rename(X.name, + '_na')


class PdFunction(TransformerMixin):
    def __init__(self, fn):
        self.fn = fn

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return self.fn(X)


class QCut(TransformerMixin):
    def __init__(self, q, interp=None):
        self.q = q
        self.interp = interp

    def fit(self, X, y=None):
        _, self.bins = pd.qcut(X, self.q, retbins=True)
        self.bin_vals = bin_interp(X, self.bins, self.interp)
        return self

    def transform(self, X):
        return (
            X
            .apply(lambda x: bin_val(x, self.bins, self.bin_vals))
            .rename(f'{X.name}_qcut{self.q}')
        )


class Rename(TransformerMixin):
    def __init__(self, name):
        self.name = name

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.rename(self.name)


class SelectColumns(TransformerMixin):
    def __init__(self, include=None, exclude=None):
        self.include = include
        self.exclude = exclude

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.include:
            X = X[self.include]
        if self.exclude:
            return X.drop(columns=self.exclude)
        return X


class SelectDtypes(TransformerMixin):
    def __init__(self, include=None, exclude=None):
        self.include = include
        self.exclude = exclude

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.select_dtypes(include=self.include, exclude=self.exclude)


class StandardScaler(TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        self.mean = X.mean()
        self.std = X.std(ddof=0)
        return self

    def transform(self, X):
        return (X - self.mean) / self.std

#Feature Engineering

### SibSp + ParCh

In [18]:
def n_fam(X):
    return (
        (X.n_sib_sp + X.n_par_ch)
        .astype(np.uint8)
        .rename('n_fam')
    )

def n_fam_2(X):
    return (
        ((X.n_sib_sp+1) * (X.n_par_ch+1))
        .astype(np.uint8)
        .rename('n_fam_2')
    )

### Cabin

In [19]:
def cabin_encode_v(X):
    return (
        (deck_encode(X) + cabin_no_encode(X) / 10)
        .astype(np.float32)
        .rename('cabin_encode_v')
    )

def cabin_encode_h(X):
    return (
        (cabin_no_encode(X) + deck_encode(X) / 10)
        .astype(np.float32)
        .rename('cabin_encode_h')
    )

def cabin_no(X):
    return (
        X
        .cabin
        .str.extract(r'(\d+)', expand=False)
        .astype(np.float32)
        .rename('cabin_no')
    )

def cabin_no_encode(X):
    def encode(x):
        if x.deck == 'T':
            return 2
        elif np.isnan(x.cabin_no):
            return np.nan
        elif x.deck == 'A':
            if x.cabin_no >= 35:
                return 4
            else:
                return 2
        elif x.deck == 'B':
            if x.cabin_no >= 51:
                return 3
            else:
                return 2
        elif x.deck == 'C':
            if x.cabin_no % 2 == 0:
                if 92 <= x.cabin_no <= 102 or 142 <= x.cabin_no <= 148:
                    return 4
                elif 62 <= x.cabin_no <= 90 or 104 <= x.cabin_no <= 140:
                    return 3
                else:
                    return 2
            else:
                if 85 <= x.cabin_no <= 93 or 123 <= x.cabin_no <= 127:
                    return 4
                elif 55 <= x.cabin_no <= 83 or 95 <= x.cabin_no <= 121:
                    return 3
                else:
                    return 2
        elif x.deck == 'D':
            if x.cabin_no >= 51:
                return 5
            else:
                return 2
        elif x.deck == 'E':
            if x.cabin_no >= 91:
                return 5
            elif x.cabin_no >= 70:
                return 4
            elif x.cabin_no >= 26:
                return 3
            else:
                return 2
        elif x.deck == 'F':
            if x.cabin_no >= 46:
                return 1
            elif x.cabin_no >= 20:
                return 5
            else:
                return 4
        elif x.deck == 'G':
            return 5
    
    df = pd.concat([X.cabin, deck(X), cabin_no(X)], axis=1)
    return (
        df
        .apply(encode, axis=1)
        .astype(np.float32)
        .rename('cabin_no_encode')
    )

def deck(X):
    return (
        X
        .cabin
        .str.extract(r'([A-Z])', expand=False)
        .rename('deck')
    )

def deck_encode(X):
    return (
        deck(X)
        .map({
            'T': 8,
            'A': 7,
            'B': 6,
            'C': 5,
            'D': 4,
            'E': 3,
            'F': 2,
            'G': 1,
        })
        .astype(np.float32)
        .rename('deck_encode')
    )

def starboard(X):
    return (
        (np.round(cabin_no(X)) % 2 == 0)
        .astype(np.uint8)
        .rename('starboard')
    )

### Name

In [20]:
def surname(X):
    return (
        X
        .name
        .str.lower()
        .str.extract(r'([a-z]+),', expand=False)
    )

def title(X):
    return (
        X
        .name
        .str.lower()
        .str.extract(r', (\w+)', expand=False)
        .rename('title')
    )

def title_fill(X):
    def rare(row):
        if row.title in ['miss', 'mrs', 'master', 'mr']:
            return row.title
        elif row.title in d:
            return d[row.title]
        elif row.sex == 'male':
            return 'mr'
        elif row.sex == 'female':
            return 'mrs'
        else:
            raise ValueError('row.sex is missing / not in [`male`, `female`]')

    miss = ['ms', 'mlle']
    mrs = ['mme', 'dona', 'lady', 'the']
    mr = [
        'capt',
        'col',
        'don',
        'jonkheer',
        'major',
        'rev',
        'sir',
    ]

    d = {
        **{k: 'mr' for k in mr},
        **{k: 'mrs' for k in mrs},
        **{k: 'miss' for k in miss}
    }

    return (
        X
        .assign(title=title)
        .apply(rare, axis=1)
        .rename('title')
    )

### Sex

In [21]:
def sex(X):
    return (
        X
        .sex
        .map({'female': 1, 'male': 0})
        .astype(np.uint8)
    )

### Ticket

In [22]:
def ticket_count(X):
    _, _, _, traintest = load_titanic()
    return count(X.ticket, traintest.ticket).astype(np.uint8)

### Interaction

In [23]:
def age_mask(X, tc, sx):
    nm = f'age_tc{tc}_sex{sx}'
    return (X.age * (X.ticket_class == tc) * (sex(X) == sx)).rename(nm)

def fare_quot(X):
    return (
        (X.fare / ticket_count(X))
        .astype(np.float32)
        .rename('fare_quot')
    )

def tc_sex(X, tc, sx):
    return (
        ((X.ticket_class == tc) & (sex(X) == sx))
        .astype(np.uint8)
        .rename(f'tc{tc}_sex{sx}')
    )

def tk_fn(X, col, fn='mean'):
    _, _, _, traintest = load_titanic()
    vc = getattr(traintest[col].groupby(traintest.ticket), fn)()
    return (
        X
        .ticket
        .apply(lambda x: vc.loc[x])
        .astype(np.float32)
        .rename(f'tk_{col}_{fn}')
    )

def tk_sex(X):
    _, _, _, traintest = load_titanic()
    vc = sex(traintest).groupby(traintest.ticket).mean()
    return (
        X
        .ticket
        .apply(lambda x: vc.loc[x])
        .astype(np.float32)
        .rename('tk_sex')
    )

Finally, the all important `surv` group of functions:

In [24]:
def surv(X):
    def encode(x):
        a = x.tk_surv_max
        b = x.sn_surv_max
        if a == 1 and b == 1:
            return 4
        elif a == 1 or b == 1:
            if a == 0 or b == 0:
                return 2
            else:
                return 3
        elif a == 0 or b == 0:
            if a == 0 and b == 0:
                return 0
            else:
                return 1
        else:
            return np.nan
    return (
        pd.concat([tk_surv(X), sn_surv(X)], axis=1)
        .apply(encode, axis=1)
        .astype(np.float32)
        .rename('surv')
    )

def sn_surv(X, fn='max'):
    tr, y, te, _ = load_titanic()
    v = getattr(y.groupby(surname(tr)), fn)()[lambda x: x.index.isin(surname(te))]
    return (
        surname(X)
        .map(v)
        .astype(np.float32)
        .rename(f'sn_surv_{fn}')
    )

def tk_surv(X, fn='max'):
    tr, y, te, _ = load_titanic()
    v = getattr(y.groupby(tr.ticket), fn)()[lambda x: x.index.isin(te.ticket)]
    return (
        X
        .ticket
        .map(v)
        .astype(np.float32)
        .rename(f'tk_surv_{fn}')
    )

# Pipeline Preprocessing

Execute

- Split `train`:
    - `train/val`: validation set `val_X, val_y`, using `VAL_SIZE`
    - `train/train`: proper train set `X, y`
- Full `train`: `tr_X, tr_y`
- Combined `train/test`: `traintest`
- `test`: as is

In [ ]:
X, y, val_X, val_y, tr_X, tr_y, test, traintest = preprocess(X_pipeline)

## Diagnostics

### Shape

In [ ]:
X.shape

### Dtypes
Check for:
- Overflow
- Column names
- Floating point error
- Anything that looks funny

In [ ]:
dtype_info(X)

Use function `vc` (value counts) to check individual columns.

### Train/Val/Test Parity

Check that each dataframe has the same dtypes and same columns, in the same order.

In [ ]:
eq_attr(X, 'columns', val_X, tr_X, test, traintest) \
    and eq_attr(X, 'dtypes', val_X, tr_X, test, traintest)

### DMatrix
XGBoost's custom data format.

In [ ]:
dtrain = xgb.DMatrix(X, y)
dval = xgb.DMatrix(val_X, val_y)
dfull = xgb.DMatrix(tr_X, tr_y)
dtest = xgb.DMatrix(test)